In [6]:
import os
import skimage
from skimage import io, transform
import math
import keras
import numpy as np
import tensorflow as tf
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D,Lambda,add,Reshape,concatenate, BatchNormalization,GlobalAveragePooling2D, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
from keras.regularizers import l2
%matplotlib inline

import keras.backend as K
K.set_learning_phase(1)

In [7]:
def se_identity_block(X, filters, r=16):
    
    F1, F2, F3 = filters
    
    X_shortcut = X
    
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid')(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    
    X = Conv2D(filters = F2, kernel_size = (3, 3), strides = (1,1), padding = 'same')(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)

    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid')(X)
    X = BatchNormalization(axis=3)(X)
    
    X_increase = X
    X = GlobalAveragePooling2D(data_format='channels_last')(X)
    X = Reshape([1,1,F3])(X)
    X = Conv2D(filters = F3//r, kernel_size = (1, 1), strides = (1,1), padding = 'valid')(X)
    X = Activation('relu')(X)
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid')(X)
    X = Activation('sigmoid')(X)
    X = layers.add([keras.layers.multiply([X,X_increase]),X_shortcut])
    X = Activation('relu')(X)

    return X

In [11]:
def se_convolutional_block(X, filters, s = 2, r=16):

    F1, F2, F3 = filters

    X_shortcut = X

    X = Conv2D(F1, (1, 1), strides = (s,s), padding='valid')(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    
    X = Conv2D(F2, (3, 3), strides = (1, 1), padding='same')(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)

    X = Conv2D(F3, (1, 1), strides = (1, 1), padding='valid')(X)
    X = BatchNormalization(axis = 3)(X)

    X_increase = X
    X = GlobalAveragePooling2D(data_format='channels_last')(X)
    X = Reshape([1,1,F3])(X)
    X = Conv2D(filters = F3//r, kernel_size = (1, 1), strides = (1,1), padding = 'valid')(X)
    X = Activation('relu')(X)
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid')(X)
    X = Activation('sigmoid')(X)
    X_proj = Conv2D(filters = F3, kernel_size = (1, 1), strides = (s,s), padding = 'valid')(X_shortcut)
    X_proj = BatchNormalization(axis = 3)(X_proj)
    X = layers.add([keras.layers.multiply([X,X_increase]),X_proj])
    X = Activation('relu')(X)
    
    return X

In [15]:
def SE_ResNet(input_shape = (224, 224,3)):
    X_input = Input(input_shape)
    X = ZeroPadding2D((3, 3))(X_input)
    X = Conv2D(64, (7, 7), strides = (2, 2))(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2),padding='same')(X)

    X = se_convolutional_block(X, filters = [64, 64, 256],  s = 1)
    X = se_identity_block(X, [64, 64, 256])
    X = se_identity_block(X, [64, 64, 256])

    X = se_convolutional_block(X, filters=[128,128,512], s = 2)
    X = se_identity_block(X, filters=[128,128,512])
    X = se_identity_block(X, filters=[128,128,512])
    X = se_identity_block(X, filters=[128,128,512])

    X = se_convolutional_block(X, filters=[256, 256, 1024], s = 2)
    X = se_identity_block(X, filters=[256, 256, 1024])
    X = se_identity_block(X, filters=[256, 256, 1024])
    X = se_identity_block(X, filters=[256, 256, 1024])
    X = se_identity_block(X, filters=[256, 256, 1024])
    X = se_identity_block(X, filters=[256, 256, 1024])

    X = se_convolutional_block(X, filters=[512, 512, 2048], s = 2)
    X = se_identity_block(X, filters=[512, 512, 2048])
    X = se_identity_block(X, filters=[512, 512, 2048])

    X = AveragePooling2D((7,7), strides=(1,1))(X)
    X = Flatten()(X)
    X = Dense(2, activation='softmax')(X)

    model = Model(inputs = X_input, outputs = X, name='ResNet50')

    return model

In [16]:
model = SE_ResNet(input_shape = (224, 224, 3))

In [17]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_4 (ZeroPadding2D (None, 230, 230, 3)  0           input_4[0][0]                    
__________________________________________________________________________________________________
conv2d_113 (Conv2D)             (None, 112, 112, 64) 9472        zero_padding2d_4[0][0]           
__________________________________________________________________________________________________
batch_normalization_73 (BatchNo (None, 112, 112, 64) 256         conv2d_113[0][0]                 
__________________________________________________________________________________________________
activation

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train,epochs = 20, batch_size = 16,validation_data=(x_test,y_test), shuffle = True)